In [1]:
import xarray as xr
import numpy as np
from bunch import Bunch

In [2]:
def readgr3(filename):
    from itertools import islice;
    out = Bunch()
    with open(filename,'r') as fid:
        # grid  name
        out.name=fid.readline().strip();        
        # number of elements and nodes
        tmp=fid.readline().split();
        out.ne=int(tmp[0]);
        out.nn=int(tmp[1]);
        # first load nodes and values 
        # not using this nn
        tmp=list(islice(fid,out.nn));
        node_id,out.x,out.y,out.z=np.loadtxt(tmp, dtype={'names':('n','x','y','z'),'formats':('i4','f8','f8','f8')}, unpack=True);
        del node_id;
        # elements
        tmp=list(islice(fid,out.ne));
        tmp_e=np.loadtxt(tmp,dtype='i4');
        out.e=tmp_e[:,2:]-1;
        fid.close();
        return out

In [6]:
hgrid = readgr3('wet_grid_jig.gr3')

In [7]:
grid = xr.Dataset(
        data_vars=dict(
        bathymetry=(["node"], hgrid.z),
        longitude=(["node"], hgrid.x),
        latitude=(["node"], hgrid.y),
        element=(["nele",  "nvertex"], hgrid.e),
        mesh_topology=(["single"], [2]),
    ),
    attrs=dict(description="Swan grid test"),
)
grid['latitude'].attrs = {'long_name': 'latitude',
                        'standard_name': 'latitude',
                        'units': 'degrees_north'}

grid['longitude'].attrs = {'long_name': 'longitude',
                         'standard_name': 'longitude',
                         'units': 'degrees_east'}

grid['element'].attrs = {
                        'long_name': 'element',
                        'standard_name': 'face_node_connectivity',
                        'units': 'nondimensional',
                        'start_index' : 0 
                        }

grid['mesh_topology'].attrs= {
                            'node_coordinates': 'longitude latitude',
                            'cf_role': 'mesh_topology',
                            'topology_dimension': 2,
                            'standard_name': "mesh_topology",
                            'long_name': "mesh topology",
                            'face_node_connectivity': "element" 
                            }

for var in ['latitude', 'longitude']:
    grid[var].encoding = {'dtype': 'f4', '_FillValue': None}

grid['bathymetry'].attrs = {
                            'long_name': 'bathymetry',
                            'standard_name': 'sea_floor_depth',
                            'units': 'm',
                            'coordinates': "latitude longitude"
                            }

for var in ['bathymetry']:

    grid[var].attrs['location'] = 'node'
    grid[var].attrs['mesh'] = 'mesh_topology'

    # chunking for faster access
    nnodes = grid['node'].size
    chunksize = [1, nnodes] if 'time' in list(grid[var].coords) else nnodes

    grid[var].encoding = {'dtype': 'f4',
                        '_FillValue': np.nan,
                        # 'zlib': True,
                        # 'complevel': 1,
                        'chunksize': chunksize}

# set some attributes to follow CF convetions for unstructured grids
# https://github.com/ugrid-conventions/ugrid-conventions



grid

<xarray.Dataset>
Dimensions:        (node: 21467, nele: 39822, nvertex: 3, single: 1)
Dimensions without coordinates: node, nele, nvertex, single
Data variables:
    bathymetry     (node) float64 18.26 18.41 18.18 ... -25.1 -25.61 -25.18
    longitude      (node) float64 115.7 115.7 115.7 115.7 ... 116.0 116.0 116.0
    latitude       (node) float64 -32.05 -32.05 -32.06 ... -32.04 -32.05 -32.04
    element        (nele, nvertex) int32 2635 2579 2585 ... 19084 19093 19075
    mesh_topology  (single) int64 2
Attributes:
    description:  Swan grid test

In [8]:
grid.to_netcdf('wet_grid_jig_qgis.nc', mode='w', format='NETCDF4')